In [1]:
import pandas as pd
import random
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from string import punctuation
from copy import deepcopy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate

""" 
TODO: Potentially optimize tokenizing by doing it parallel so larger dataset can be created
"""

' \nTODO: Potentially optimize tokenizing by doing it parallel so larger dataset can be created\n'

In [3]:
"""
Dataset from: https://www.kaggle.com/datasets/carlosgdcj/genius-song-lyrics-with-language-information?resource=download
Size: 9.07 GB

Since the data is too large to use we can sample from it using p to approximately represent the % of data we will take.

This works by generating a number in the interval of 0-1 and if that value is greater than p the line will be skipped,
approximating a random sample.

"""

file_path = 'song_lyrics.csv'
p = 0.1  
df = pd.read_csv(
         file_path,
         header=0, 
         skiprows=lambda i: i>0 and random.random() > p
)

In [42]:
"""
For the inital stages we will cutoff the sampled df above to include a good distribution of each genre and be small enough
that we can verify our proccess works.
"""

SAMPLE = deepcopy(df.groupby('tag').head(1000))

In [43]:

"""
Here is just tokenizing/removing stop words from the lyrics and preparing the data for EDA / training
"""
stop_words = set(stopwords.words('English'))
punctuation = set(punctuation)

def Clean(text):
    tokens = word_tokenize(text)
    clean_tokens = [token.lower() for token in tokens if token.lower() not in stop_words and token.lower() not in punctuation and token.isalpha()]
    return clean_tokens

SAMPLE['lyrics'] = SAMPLE['lyrics'].apply(Clean)
data_list = []
data_list.append(SAMPLE['lyrics'])
sentences = SAMPLE['lyrics'].values
SAMPLE['lyrics'] = [' '.join(doc) for doc in sentences]
SAMPLE.head()
SAMPLE.to_csv("SAMPLE_out.csv")

,title,tag,artist,year,views,features,lyrics,id,language_cld3,language_ft,language
0,Can I Live,rap,JAY-Z,1996,468624,{},produced irv gotti intro yeah hah yeah invite ...,3,en,en,en
1,Lollipop Remix,rap,Lil Wayne,2008,580832,"{""Kanye West"",""Static Major""}",intro lil wayne haha homo young mula baby say ...,7,en,en,en
2,Losing Weight Pt. 2,rap,Cam'ron,2002,32712,"{""Cam\\'ron"",""Juelz Santana""}",chorus ayo fuck losing weight back highways mo...,19,en,en,en
3,Youre Nobody Til Somebody Kills You,rap,The Notorious B.I.G.,1997,302936,{},produced jiv poss puff daddy stevie j dj enuff...,45,en,en,en
4,Major Pain Bossalini,rap,C-Bo,1998,442,"{""Spice 1""}",verse got warehouse packed lamborginis lacs pe...,12292,en,en,en


In [2]:
"""
This is a quick test to verify the lyrics are prepared to be used for training, using countvectorizer
"""
SAMPLE = pd.read_csv('SAMPLE_out.csv')
x_train, x_test, y_train, y_test = train_test_split(SAMPLE['lyrics'], SAMPLE['tag'], test_size=0.2)
print("Split Created")
vectorizer = CountVectorizer()
vectorizer.fit(x_train)
print("Vectorized")
X_training = vectorizer.transform(x_train)
X_testing = vectorizer.transform(x_test)
print("Transformed")
RF = RandomForestClassifier()
RF.fit(X_training, y_train)
print("Fitted")
y_pred = RF.predict(X_testing)

accuracy_test = accuracy_score(y_test, y_pred)
print(accuracy_test)

Split Created
Vectorized
Transformed
Fitted
0.6041666666666666
